In [75]:
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint
from datetime import date, timedelta
import pytz
from collections import deque
import math
import talib

In [76]:
stock_x = 0
stock_y = 0
def initialize(context):
    
    global stock_x
    global stock_y
    
    set_commission(commission.PerShare(cost=0.03, min_trade_cost=None))
    set_slippage(slippage.FixedSlippage(spread=1.00))
    stock_pairs = [(sid(32304), sid(33334)),
                           (sid(27894),sid(32307))]
    
    pair = 2
    
    if pair == 1:
        stock_x, stock_y = stock_pairs[0]
    if pair == 2:
        stock_x, stock_y = stock_pairs[1]
    
    context.initial_setup = False
    context.entry_amount = 1000
    context.entry_threshold = 1.5
    context.exit_threshold = 0.0
    context.lookback = 20
    context.z_window = 20
    context.adf_threshold = 0.05
    

    schedule_function(trade, date_rules.every_day(), time_rules.market_close(minutes=60))
    set_symbol_lookup_date('2010-01-01')
                          
                        

In [77]:
def check_pair_status(x, y, context):
    
    if coint(x,y)[1] < context.adf_threshold:
        return True
    else:
        return False

In [78]:
def build_model(context,data):
    global stock_x
    global stock_y
    

    x_history = data.history(stock_x, 'price', context.lookback, '1d')
    y_history = data.history(stock_y, 'price', context.lookback, '1d')
    x = np.array(x_history)
    X = sm.add_constant(x)
    y = np.array(y_history)
    model = sm.OLS(y, X)
    results = model.fit()
    context.beta_1, context.const = results.params
        
    # Get y_hat
    y_hat = context.beta_1 + context.const * x
        
    # Get the spread
    spread = y - y_hat
        
    # Get standard deviation of spread
    context.spread_std = np.std(spread)
    
    context.is_cointegrated = check_pair_status(x, y, context)

In [79]:
def get_current(context,data):
    global stock_x
    global stock_y
    
    current_spread = data.current(stock_y, 'price') - (context.beta_1 + context.const * data.current(stock_x, 'price'))
    z_score = current_spread / context.spread_std
    return current_spread, z_score


In [80]:
def trade (context, data):
    global stock_x
    global stock_y
    
    if get_open_orders(stock_x) or get_open_orders(stock_y):
        return
    
    stock_x_price = data.current(stock_x, 'price')
    stock_y_price = data.current(stock_y, 'price')
    
    if context.initial_setup == False:
        build_model(context, data)
        
    current_spread, z_score = get_current(context, data)
    
    equilibrium = np.copysign(1, z_score)
    
    #Exit trade if the pair has reached equilibrium
    if len(context.portfolio.positions) > 0 and np.any(equilibrium != context.entry_sign or abs(z_score) < context.exit_threshold):
        order_target_percent(stock_x, 0)
        order_target_percent(stock_y, 0)
    #Enter trade  
    if len(context.portfolio.positions) == 0:
        #rebuild model
        build_model(context,data)
        get_current(context,data)
        #check equilibrium
        equilibrium = np.copysign(1, z_score)
        #check for cointegration and spread
        if (context.is_cointegrated and abs(z_score) >= context.entry_threshold):
            #reset relationship to equilibrium
            context.entry_sign = equilibrium
            #compute shares and shares to buy
            shares_x = round(context.entry_amount / stock_x_price, 0)
            shares_y = round(context.entry_amount / stock_y_price, 0)
            
            order(stock_x,      equilibrium * shares_x)
            order(stock_y, -1 * equilibrium * shares_y)
            
    
    
    
    